# Building a Currency Neutral Portfolio #

In [105]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Plots
import matplotlib.pyplot as plt

# Statistics
import statsmodels.api as sm
from scipy.optimize import minimize

# Import Data
import yfinance as yf

# Manipulate Files
import os

# Pretty Notation
from IPython.display import display, Math

In [106]:
def import_financial_data(
    ticker: str
):

    # Check the ticker for Upper Cases
    ticker = ticker if ticker.isupper() else ticker.upper()

    # Import data
    df = pd.read_csv(rf"..\stocks\{ticker}.csv")

    # Set the Index
    df = df.set_index('Date')
    df.index = pd.to_datetime(df.index)

    df_useful_data = df[['Open Price', 'High Price', 'Low Price', 'Close Price', 'Adjusted_close']]

    df_useful_data = df_useful_data.rename(columns={
        "Open Price":"open",
        "High Price":"high",
        "Low Price":"low",
        "Close Price":"close",
        "Adjusted_close":"adjusted_close",
    })

    # Drop NaN's
    df_useful_data.dropna(inplace = True)

    return df_useful_data.loc["2015-01-01":]

In [107]:
# Create the Weights function
def wexp(N, half_life):
    c = np.log(0.5)/half_life
    n = np.array(range(N))
    w = np.exp(c*n)
    return np.flip(w/np.sum(w))

In [108]:
# First Call the Exchange Rate Index
data = yf.download('DX-Y.NYB', start='2015-01-01', end='2025-01-01', interval='1d', auto_adjust=True)
data.dropna(inplace=True)

# Close Price

dxy_data = data['Close']['DX-Y.NYB']
dxy_data

In [109]:
# Create Plot

plt.figure(figsize=(10, 6))
plt.plot(dxy_data, label='DXY Dollar Index', color='red', alpha=0.7)

# Config
plt.title('DXY Dollar Inde Time Series')
plt.xlabel('Time')
plt.ylabel('Index')
plt.legend()

# Show
plt.show()

In [110]:
# Get the important data for the Risk Free Rate
rfr = pd.read_csv(r"..\additional_data\rfr.csv")
rfr = rfr.set_index('Date')
rfr.index = pd.to_datetime(rfr.index, dayfirst=True)
rfr.dropna(inplace = True)

# Get the important data for the S&P500
sp500 = pd.read_csv(rf"..\additional_data\sp500.csv")
sp500 = sp500.set_index('Date')
sp500.index = pd.to_datetime(sp500.index)

In [111]:
# Create the Returns
benchmark_returns = sp500['sp_500'].pct_change(1).dropna()
dollar_returns = dxy_data.pct_change(1).dropna()
risk_free_daily = (((1 + (rfr['risk_free_rate'].div(100)))**(1/360)) - 1)

# Create the Excess Returns
benchmark_excess = benchmark_returns - risk_free_daily
dollar_excess = dollar_returns - risk_free_daily

In [112]:
# Create Figure
fig, ax1 = plt.subplots(dpi = 300)

# Market Returns Plot
benchmark_excess.cumsum().plot(color = 'blue', ax = ax1, alpha=0.5, label = 'Market')
ax1.set_xlabel('Date')
ax1.set_ylabel(
    'Market Returns', 
    color='blue'
    )

# Dollar Returns Plot
ax2 = ax1.twinx()

dollar_excess.cumsum().plot(color = 'red', ax = ax2, alpha=0.8, label = 'Dollar Index')
ax2.set_ylabel(
    'Dollar Returns', 
    color='red'
    )

plt.title('Returns vs Returns Time Series')
plt.show()

In [113]:
# Market and Dollar Correlations

market_dollar_corr = dollar_excess.corr(benchmark_excess)

market_dollar_corr

In [114]:
# Import Data

# XOM Data
df_1 = import_financial_data('XOM')

# LMT Data
df_2 = import_financial_data('LMT')

# NOC Data
df_3 = import_financial_data('NOC')

# GS Data
df_4 = import_financial_data('GS')

# GD Data
df_5 = import_financial_data('GD')

In [115]:
# Let us find an unsensible portfolio to the market

data_portfolio = pd.DataFrame()

data_portfolio['XOM'] = df_1['adjusted_close']
data_portfolio['LMT'] = df_2['adjusted_close']
data_portfolio['NOC'] = df_3['adjusted_close']
data_portfolio['GS'] = df_4['adjusted_close']
data_portfolio['GD'] = df_5['adjusted_close']

data_portfolio = data_portfolio.dropna()

data_portfolio = data_portfolio.loc['2015-01-01':]

data_portfolio

In [116]:
# Create the Returns

df_returns = data_portfolio.pct_change(1).dropna() 
df_returns = df_returns.subtract(risk_free_daily, axis=0)
df_returns.dropna(inplace = True)

df_returns

In [117]:
# Check correlations with dollar returns

df_returns['dollar_returns'] = dollar_excess
df_returns['market_returns'] = benchmark_excess

df_returns.corr()

In [118]:
# Create Plot

plt.figure(figsize=(10, 6))
plt.plot(df_returns.cumsum(), label=df_returns.columns, alpha=0.7)

# Config
plt.title('Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Index')
plt.legend()

# Show
plt.show()

### Use CAPM Residuals ###

In [120]:
# Get the important data for the S&P500
capm_residuals = pd.read_csv(r"..\additional_data\capm_residuals.csv")
capm_residuals = capm_residuals.set_index('Date')
capm_residuals.index = pd.to_datetime(capm_residuals.index)
capm_residuals = capm_residuals[data_portfolio.columns]

# Add the Dollar Returns
capm_residuals['dollar_returns'] = dollar_excess

capm_residuals

In [121]:
plt.figure(figsize=(10, 6))
plt.plot(capm_residuals.cumsum(), label=capm_residuals.columns, alpha=0.7)

# Config
plt.title('Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Index')
plt.legend()

# Show
plt.show()

In [122]:
# Correlations

capm_residuals.corr()

### Calculate the Betas ###

In [124]:
# Lets calculate the betas for the DXY

tickers = []
betas = []
lower_bounds = []
upper_bounds = []

# Calculate Weights
window = len(df_returns)
weights = window * wexp(window, window/2)

for stock in data_portfolio.columns:
    #Model specification
    model = sm.WLS(
        df_returns[stock], 
        sm.add_constant(df_returns[['market_returns', 'dollar_returns']]),
        missing='drop',
        weights=weights,
        )   
         
    #the results of the model
    results = model.fit()

    # Get the Parameters and CI
    params = results.params
    ci = results.conf_int(alpha=0.05)  # 95% CI
    
    tickers.append(stock)
    betas.append(params.iloc[2])
    lower_bounds.append(ci.iloc[2][0])  # First column: lower bound
    upper_bounds.append(ci.iloc[2][1])  # Second column: upper bound


In [125]:
# Example data: beta estimates and confidence intervals
betas_intervals = pd.DataFrame({
    'beta': betas,
    'ci_lower': lower_bounds,
    'ci_upper': upper_bounds,
}, index=tickers)

betas_intervals

In [154]:
# Calculate symmetric errors for error bars
errors = [betas_intervals['beta'] - betas_intervals['ci_lower'], betas_intervals['ci_upper'] - betas_intervals['beta']]

# Create the plot
fig, ax = plt.subplots(figsize=(8, 5))

ax.errorbar(
    x=betas_intervals['beta'],                   # x-values (betas)
    y=range(len(betas_intervals)),               # y-positions
    xerr=errors,                      # confidence interval errors
    fmt='o',                          # circular markers for betas
    ecolor='gray',                    # color of the error bars
    capsize=5,                        # small caps on error bars
    elinewidth=2,                    # thickness of the error bars
    markeredgewidth=2                # thickness of the circle edge
)

# Customize the plot
ax.set_yticks(range(len(betas_intervals)))
ax.set_yticklabels(betas_intervals.index)
ax.axvline(0, color='red', linestyle='--')  # reference line
ax.set_xlabel('Beta estimate')
ax.set_title('Betas with Confidence Intervals')

plt.tight_layout()
plt.show()


In [127]:
# Define the Function
def dxy_betas(
    stock_excess,
    benchmark_excess = benchmark_excess,
    dollar_excess = dollar_excess,
):
    # Common Index
    common_index = stock_excess.index.intersection(benchmark_excess.index).intersection(dollar_excess.index)
    common_index = sorted(common_index)

    # Reindex
    stock_excess = stock_excess.loc[common_index]
    benchmark_excess = benchmark_excess.loc[common_index]
    dollar_excess = dollar_excess.loc[common_index]
    
    # Create the data
    y = stock_excess
    x = pd.concat([benchmark_excess, dollar_excess], axis=1)
    
    # Rolling window
    window = 252
    weights = wexp(window, window/2)
    
    betas = []
    index = []
    
    for i in range(window, len(y)):
        Y_window = y.iloc[i - window:i]
        X_window = x.iloc[i - window:i]
        
        X_window = sm.add_constant(X_window)
        
        model = sm.WLS(Y_window, X_window, missing='drop', weights=weights).fit()
        
        betas.append(model.params.values)
        index.append(y.index[i])

    parameters_df = pd.DataFrame(betas, columns=["const", "capm_betas", "dxy_betas"], index=index)

    return parameters_df

In [128]:
# Get the betas
dxy_betas_df = pd.DataFrame()

# Define the loop
for stock in data_portfolio.columns:
    df = dxy_betas(df_returns[stock])

    dxy_betas_df[stock] = df['dxy_betas']

# See betas
dxy_betas_df

In [129]:
plt.figure(figsize=(10, 6))
plt.plot(dxy_betas_df.ewm(span=21, adjust = False).mean(), label=dxy_betas_df.columns, alpha=0.7)
plt.axhline(y=0, color='black', linestyle='dashed')

# Config
plt.title('DXY Betas Time Series')
plt.xlabel('Time')
plt.ylabel('Betas')
plt.legend()

# Show
plt.show()

### Portfolio Optimization ###

In [258]:
# Let us Calculate the Weights
def rolling_weights(
    returns, 
    betas, 
    window=252, 
    rebalance_freq=63
):
    # Common Index
    common_index = returns.index.intersection(betas.index)  # Common Date
    returns = returns.reindex(common_index)
    betas = betas.reindex(common_index)

    # Lists to Store Things
    weights_list = []
    dates = []

    for i in range(window, len(returns), rebalance_freq):
        past_returns = returns.iloc[i-window:i]  # Rolling Window
        past_betas = betas.iloc[i-window:i]
        
        # Mean and Covariance
        beta = past_betas.mean()
        Sigma = past_returns.cov()

        # Inverse
        lambda_ = 1e-6  # Tikhonov Regularization
        Sigma_inv = np.linalg.inv(Sigma + lambda_ * np.eye(Sigma.shape[0]))
        
        # Sigma_inv = np.linalg.inv(Sigma)

        # Ones
        iota = np.ones(len(beta))

        # And now obtain the coefficients
        C = np.dot(np.dot(iota.T, Sigma_inv), iota)
        D = np.dot(np.dot(beta.T, Sigma_inv), beta)
        E = np.dot(np.dot(beta.T, Sigma_inv), iota)
        Delta = (D*C - E*E)
        
        w = ((D/Delta)*(Sigma_inv @ iota)) - ((E/Delta)*(Sigma_inv @ beta))

        # Save weights and dates
        weights_list.append(w)
        dates.append(returns.index[i])

    # Create the DataFrame
    weights_df = pd.DataFrame(weights_list, index=dates, columns=returns.columns)

    # Expand the DataFrame
    weights_df = weights_df.reindex(returns.index, method='ffill')

    return weights_df.dropna()

In [260]:
# Obtain the Weights

cnp_weights = rolling_weights(df_returns[dxy_betas_df.columns], dxy_betas_df)

cnp_weights

In [262]:
# Calculate the ZBP

cnp_returns = ((df_returns[dxy_betas_df.columns] * cnp_weights).dropna()).sum(axis = 1)
cnp_returns.name = 'CNP'

cnp_returns

In [264]:
# Create Plot

plt.figure(figsize=(10, 6))
plt.plot(cnp_returns.mul(100).cumsum(), label='CNP Returns', color='blue', alpha=0.7)
plt.axhline(y=0, color='black', linestyle='dashed')

# Config
plt.title('CNP Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()

# Show
plt.show()

In [266]:
# Create Plot

plt.figure(figsize=(10, 6))
plt.plot(cnp_returns.cumsum(), label='Currency-Neutral Portfolio Returns', color='red', alpha=0.7)
plt.plot(dollar_returns.loc['2017':].cumsum(), label='Dollar Returns', color='blue', alpha=0.7)
plt.axhline(y=0, color='black', linestyle='dashed')

# Config
plt.title('Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()

# Show
plt.show()

In [268]:
def calculate_analytics(df_returns, risk_free_rate=0.0):
    # Trading Days in one Year
    ann_factor = 252  
    
    # Annualized Returns
    annualized_return = df_returns.mean() * ann_factor
    
    # Annualized Volatility
    annualized_std = df_returns.std() * np.sqrt(ann_factor)
    
    # Sharpe Ratio
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_std
    
    # Max Drawdown
    cumulative_returns = (1 + df_returns.div(100)).cumprod()
    rolling_max = cumulative_returns.cummax()
    drawdown = (cumulative_returns / rolling_max) - 1
    max_drawdown = drawdown.min()

    # VaR at 95%
    var_95 = df_returns.quantile(0.05)

    # Create DF
    summary_df = pd.DataFrame({
        "Annualized Returns": annualized_return,
        "Annualized Volatility": annualized_std,
        "Sharpe Ratio": sharpe_ratio,
        "Max Drawdown": max_drawdown,
        "VaR 95%": var_95
    })
    
    return summary_df

In [270]:
# Check the Analytics

df_analytics = df_returns[dxy_betas_df.columns].copy()
df_analytics['CNP_Port'] = cnp_returns

df_analytics.dropna(inplace=True)

df_analytics

In [272]:
# And the Table

analytics_table = calculate_analytics(df_analytics)

analytics_table

In [274]:
# Correlation with the DXY

dollar_returns.corr(cnp_returns)